# PyTorch Tutorial

This tutorial is based on [Deep Learning with PyTorch: A 60 Minute Blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html).

PyTorch is a Python-based scientific computing package targeted at two sets of
audiences:

* A replacement for NumPy to use the power of GPUs
* A deep learning research platform that provides maximum flexibility
   and speed


## Getting Started

### Tensor Construction

* Tensors are similar to NumPy's ndarrays, with the addition being that
* Tensors can also be used on a GPU to accelerate computing
* Commonly, vectors / matrices with more than 2 dimensions are called tensors

In [ ]:
import torch

Construct a 5x3 matrix, uninitialized:



In [ ]:
x = torch.empty(5, 3)
print(x)

Construct a randomly initialized matrix:



In [ ]:
x = torch.rand(5, 3)
print(x)

Construct a matrix filled zeros and of dtype long:



In [ ]:
x = torch.zeros(5, 3, dtype=torch.long) # ones
print(x)

Construct a tensor directly from data:



In [ ]:
x = torch.tensor([5.5, 3])
print(x)

Get its size:



In [ ]:
print(x.size())
print(x.shape)

### Basic Operations

PyTorch support most of the NumPy operations

In [ ]:
x = torch.rand(5, 3)
y = torch.rand(5, 3)
# return a new tensor
print(x + y) # torch.add(x, y)

In-place addition uses following function



In [ ]:
# adds x to y
y.add_(x)
print(y)

<div class="alert alert-info"><h4>Note</h4><p>Any operation that mutates a tensor in-place is post-fixed with an ``_``.
    For example: ``x.copy_(y)``, ``x.t_()``, will change ``x``.</p></div>

You can use standard NumPy-like indexing with all bells and whistles!



In [ ]:
print(x[:, 1])

<font color="red"><b>Resizing</b></font>: If you want to resize/reshape tensor, you can use ``torch.view``:



In [ ]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

The `unsqueeze` function acts like `np.newaxis` but is more convenient. The `squeeze` function does the opposite.

In [ ]:
x = torch.rand(4, 3)
print(x.shape)
x = x.unsqueeze(0) # insert a dimension at axis 0
print(x.shape)
x = x.squeeze(0) #  delete a dimension at axis 0
print(x.shape)

Remember the `axis` keyword in NumPy. PyTorch also has this kind of clustering functions like `sum` and `mean`, but with input parameter `dim`.

In [ ]:
x = torch.rand(5, 2)
print(x)
print(x.sum(dim=0))
print(x.sum(dim=1))

If you have a one element tensor, use ``.item()`` to get the value as a
Python number



In [ ]:
x = torch.randn(1)
print(x)
print(x.item())

### NumPy Bridge

Converting a Torch Tensor to a NumPy array and vice versa is a breeze.

The Torch Tensor and NumPy array will share their underlying memory
locations (if the Torch Tensor is on CPU), and changing one will change
the other.

#### Converting a Torch Tensor to a NumPy Array



In [ ]:
a = torch.ones(5)
print(a)

In [ ]:
b = a.numpy()
print(b)

See how the numpy array changed in value.

Be careful of **sharing references**!


In [ ]:
a.add_(1)
print(a)
print(b)

#### Converting NumPy Array to Torch Tensor

See how changing the np array changed the Torch Tensor automatically



In [ ]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

All the Tensors on the CPU except a CharTensor support converting to
NumPy and back.


### CUDA Tensors

Tensors can be moved onto any device using the ``.to`` method.



In [ ]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!



## [Autograd: Automatic Differentiation](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#autograd-automatic-differentiation)


Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our
first neural network.


The ``autograd`` package provides automatic differentiation for all operations
on Tensors. It is a define-by-run framework, which means that your backprop is
defined by how your code is run, and that every single iteration can be
different.

Let us see this in more simple terms with some examples.


### Tensor

``torch.Tensor`` is the central class of the package. If you set its attribute
``.requires_grad`` as ``True``, it starts to track all operations on it. When
you finish your computation you can call ``.backward()`` and have all the
gradients computed automatically. The gradient for this tensor will be
accumulated into ``.grad`` attribute.

To stop a tensor from tracking history, you can call ``.detach()`` to detach
it from the computation history, and to prevent future computation from being
tracked. (`.detach()` may be very useful when implementing RNN)

To prevent tracking history (and using memory), you can also wrap the code block
in ``with torch.no_grad():``. This can be particularly helpful when evaluating a
model because the model may have trainable parameters with
``requires_grad=True``, but for which we don't need the gradients.

There’s one more class which is very important for autograd
implementation - a ``Function``.

``Tensor`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each tensor has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Tensor`` (except for Tensors created by the user - their
``grad_fn is None``).

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Tensor``. If ``Tensor`` is a scalar (i.e. it holds a one element
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``gradient``
argument that is a tensor of matching shape.



Create a tensor and set ``requires_grad=True`` to track computation with it



In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

Do a tensor operation:

In [ ]:
y = x + 2
print(y)

You may find `y` also has ``grad_fn``. Let's do more operations on `y`:

In [ ]:
z = y * y * 3
out = z.mean()

print(z, out)

Let's firstly visualize the **computational graph** (need to install torchviz and graphviz first)
```
from torchviz import make_dot
make_dot(out)
```

![](fig/computational_graph.jpg)

Let's backprop now.
Because ``out`` contains a single scalar, ``out.backward()`` is
equivalent to ``out.backward(torch.tensor(1.))``.

In [ ]:
out.backward()

Print gradients $\frac{\partial (out)}{\partial \mathbf{x}}$



In [ ]:
print(x.grad)

You should have got a matrix of ``4.5``.

Denote the ``out``*Tensor* $o$.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.